In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import networkx as nx
import numpy as np
from lib.graph_local_classes import GraphStructure, GraphParams, InnerGraphSimulation
f

In [5]:
logmean np.random.random(size=(20,5))

array([[ 0.50648813,  0.72048957,  0.76308834,  0.75717963,  0.1149179 ],
       [ 0.29919279,  0.78909145,  0.55136613,  0.86529601,  0.36335707],
       [ 0.38205836,  0.45532305,  0.83391321,  0.33104917,  0.11771307],
       [ 0.40278419,  0.70901091,  0.15038111,  0.27407319,  0.737474  ],
       [ 0.95059704,  0.90341016,  0.74156563,  0.57058762,  0.77340271],
       [ 0.16253126,  0.92707191,  0.24444899,  0.3007158 ,  0.43584605],
       [ 0.27280878,  0.0843301 ,  0.05195333,  0.87137782,  0.86584603],
       [ 0.19827532,  0.72720921,  0.87086757,  0.31222356,  0.84651116],
       [ 0.72305811,  0.98759717,  0.8532401 ,  0.70018914,  0.9447325 ],
       [ 0.95102745,  0.18653906,  0.16125501,  0.86090777,  0.33977376],
       [ 0.53120436,  0.45225128,  0.2428219 ,  0.85155478,  0.50098636],
       [ 0.06182055,  0.35532362,  0.62963839,  0.2663882 ,  0.72946442],
       [ 0.48306606,  0.14261525,  0.89895772,  0.58583186,  0.10955857],
       [ 0.82816037,  0.44278248,  0.6